**Neural Network Tester and Trainer**

# Imports and Initializations

In [ ]:
"""
  This ipynb file is responsible to train your model. Please prepare a .npy file beforehand to train it.
  GrayScaled or RGB images are accepted.
"""

import time
from tqdm import tqdm
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import matplotlib.pyplot as plt
from matplotlib import style
from math import ceil
import torch.nn.functional as F
import os
from sys import path as sysPath

# Set your CWD path to folder containing DL Model Architecture.
os.chdir(sysPath[0])
print("CWD: ", os.getcwd())

# NETWORK ARCHITECTURE TO IMPORT [inside myModel folder following the example on top]
import COV_NET5 as NET_CLASS
import importlib

# Reload Depp Learning model if there's any changes to its architecture
importlib.reload(NET_CLASS)

print("GPU Avaialble:", torch.cuda.device_count())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Net = NET_CLASS.Neural().to(device)
Net.showNetworkDetails()

# Prepare Training Data

In [ ]:
import random
import cv2 as cv

# initialize pseudo-random generator
random.seed()

# == Training Data Section ==

# set it to 3 dimension for RGB / BGR images
CHANNELS = 3

GRAY = True # if you're training GRAYSCALED images, set True

if GRAY:
  # set it to 1 dimension
  CHANNELS = 1
  
# IMAGE W/H and CLASSIFICATION LABELS
IMG_WIDTH = NET_CLASS.IMG_WIDTH
IMG_HEIGHT = NET_CLASS.IMG_HEIGHT
LABELS = NET_CLASS.CLASS_LABELS

# Data augmentation to add. Currently only padding and rotation. [augment as much as you can]
data_transforms = transforms.Compose([
        transforms.Pad(random.randrange(10, 34, 2)),
        transforms.Resize((IMG_HEIGHT,IMG_WIDTH)),
        transforms.RandomRotation(degrees=[-15,+15])
])

# TRAINING DATA TO IMPORT. EXP: training_data.npy
training_data = np.load("", allow_pickle=True)

# Shuffle and randomize the data loaded to encourage generalization and proper convergence.
np.random.shuffle(training_data)

y_all = []
x_all = []

for i in training_data:
        y_all.append(i[1].tolist())
y_all = torch.Tensor(np.array(y_all))


# === Create x_all training data for BGR ===

if not GRAY:

    for i in training_data:
        x_data = np.transpose(i[0],(2,0,1)) # (3, H, W)        
        x_all.append(x_data)

    # convert to cuda float tensors
    x_all = torch.cuda.FloatTensor(x_all) 

    # augment data with 80% of being chosen
    for i in tqdm(range(len(x_all))):
        if random.random() < 0.8:
            x_all[i] = data_transforms(x_all[i])

# === Create x_all training data for BGR ===


# === Create x_all training data for GRAYSCALED ===

else:
    # convert data to torch.FloatTensor
    x_all = torch.Tensor([i[0] for i in training_data])

    # augment data with 80% of being chosen
    for i in tqdm(range(len(x_all))):
        if random.random() < 0.8:
            x = x_all[i]
            
            x_all[i] = data_transforms(x.view(1, IMG_HEIGHT, IMG_WIDTH))

# === Create x_all training data for GRAYSCALED ===


# Make all GRAYSCALED or BGR values range from 0-255 to 0-1 [normalize inputs]
x_all *= (1/255)

# Separate loaded data into training and testing
VAL_PCT = 0.1
val_size = ceil(len(x_all)*VAL_PCT)

train_X = x_all[:-val_size]
train_y = y_all[:-val_size]

test_X = x_all[-val_size:]
test_y = y_all[-val_size:]

print("total len:", len(x_all))
print("train len:", len(train_X))
print("test len:", len(test_X))

Show Training Data's Features

In [ ]:
# Show the shape of tensors loaded for confirmation
print(test_X[4].shape)
print(test_X[4].view(-1, CHANNELS, IMG_HEIGHT,IMG_WIDTH).shape)
print(test_X[4:8].shape)
print(test_X[4:8].view(-1, CHANNELS, IMG_HEIGHT, IMG_WIDTH).shape)

# Get random test image
NUM = random.randint(0, len(test_X))
image = test_X[NUM]

# Show images here
if not GRAY:
    # RGB IMG
    image = image.cpu().permute(1, 2, 0).numpy()[...,::-1].copy()

    print(image.shape)
    print("Y:", test_y[NUM])
    plt.imshow(image)

else:
  # GRAY IMG
  print(image.shape)
  print("Y:", test_y[NUM])
  plt.imshow(image, cmap="gray")


# Training


In [ ]:
# Choose optimizer to use
optimizer = optim.Adam(Net.parameters(), lr=0.00001)
# optimizer = optim.SGD(Net.parameters(), lr=0.001, momentum=0.8)

# Choose loss function
loss_function = nn.MSELoss()


def fwd_pass(X,y,train=False):
    if not train:
        Net.zero_grad()
    
    outputs = Net(X)

    matches = []
    matches = [torch.argmax(i)==torch.argmax(j) for i,j in zip(outputs, y)]
    acc = matches.count(True) / len(matches)    
    
    loss = loss_function(outputs, y)
    
    if train:
        loss.backward()
        optimizer.step()
    
    return acc, loss

def test(size=32):
    
    random_start = np.random.randint(len(test_X)-size)
    X, y = test_X[random_start:random_start+size], test_y[random_start:random_start+size]
    
    with torch.no_grad():
        val_acc, val_loss = fwd_pass(X.view(-1, CHANNELS, IMG_HEIGHT,IMG_WIDTH).to(device), y.to(device))
    
    return val_acc, val_loss

def trainModel():
    BATCH_SIZE = 16
    TEST_BATCH_SIZE = 32
    EPOCHS = 13
    
    with open("currentModelLog.log", "w") as f:
        for epoch in range(EPOCHS):
            print(f"EPOCH {epoch + 1}", end=": ")
            
            for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
                batch_X = train_X[i:i+BATCH_SIZE].view(-1, CHANNELS, IMG_HEIGHT,IMG_WIDTH).to(device)
                batch_y = train_y[i:i+BATCH_SIZE].to(device)
                
                # Accuracy and loss for train images
                acc, loss = fwd_pass(batch_X, batch_y, train=True)
                
                # for every 25 steps optimizer take, use test batches
                if i % 25 == 0:

                    # Accuracy and loss from test images
                    val_acc, val_loss = test(TEST_BATCH_SIZE)
                    f.write(f"{round(time.time(),3)}, { round(float(acc),2) }, {round(float(loss),2)}, {round(float(val_acc),2)}, {round(float(val_loss),2)}\n")
trainModel()

In [ ]:
# Create graph to show training and testing's accuracy and loss values
style.use("ggplot")

def createGraph():
    contents = open("currentModelLog.log", "r").read().split("\n")
    
    times = []
    accuracies = []
    losses = []
    
    val_accs = []
    val_losses = []

    for c in contents:
        if c:
            timestamp, acc, loss, val_acc, val_loss = c.split(",")
            
            times.append(float(timestamp)) 
            accuracies.append(float(acc)) 
            losses.append(float(loss)) 
            val_accs.append(float(val_acc)) 
            val_losses.append(float(val_loss)) 
    
    ax1 = plt.subplot2grid((2,1), (0,0))
    ax2 = plt.subplot2grid((2,1), (1,0))
    # ax2 = plt.subplot2grid((2,1), (1,0), sharex=ax1)
    
    ax1.plot(times, accuracies, label="T Accuracy")
    ax1.plot(times, val_accs, label="V Accuracy")
    ax1.legend(loc=2)
    
    ax2.plot(times, losses, label="T Loss")
    ax2.plot(times, val_losses, label="V Loss")
    ax2.legend(loc=2)
    
    plt.show()
    
createGraph()


Save trained weights

In [ ]:
# provide directory and .pth file name to save trained weights.
torch.save(Net.state_dict(), "")

Test the trained weights of the model on a random image from test_X

In [ ]:
TEST_NUM = random.randint(0, len(test_X))

answer = -1

prediction = Net(test_X[TEST_NUM].view(-1, CHANNELS, IMG_HEIGHT, IMG_WIDTH).to(device))[0]

print("PREDICTION: ", prediction)

for j in prediction:
    # if at least one prediction is above 10%
    if(j.item() > 0.1):
        
        answer = torch.argmax(prediction).item()
        prob = prediction[answer].item()
        break

print("ANSWER INDEX:", answer)
print("ANSWER PROBABILITY:", prob)

if answer < 0:
    print("ANSWER LABEL:", "UNKNOWN")
else:
    print("ANSWER LABEL:", LABELS[answer])

image = test_X[TEST_NUM]

print("PLT IMG SHAPE: ", image.shape)

if not GRAY:
    # RGB IMG
    image = image.cpu().permute(1, 2, 0).numpy()[...,::-1].copy()
    plt.imshow(image)

else:
    # GRAY IMG
    plt.imshow(image, cmap="gray")